## Corpus Bot Conjugador

Este es un pequeño corpus que contiene dos novelas contemporáneas escritas en lituano. A partir de estos dos textos se crea una colección de unidades léxicas no anotadas y se extraen las formas susceptibles de ser verbos, se comprueba su existencia en el diccionario online de la lengua lituana y se extraen las tres formas verbales principales (infinitivo, presente y pasado) para el posterior uso del bot.

In [1]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [4]:
# Es el directorio del corpus.
corpusdir = "/home/mgrabyte/corpus_lt"

newcorpus = PlaintextCorpusReader(corpusdir, '.*')
palabras_corpus = newcorpus.words()

In [5]:
# A partir de una palabra tratamos de construir un infinitivo para luego comprobar si 
# existe en el diccionario online.
def regla_infinitivo(palabra):
    """Construye una forma del infinitivo"""
    if palabra[-2:] == "ti":
        infinitivo = palabra
    elif palabra[-1] == "a" and not palabra[-2:] == "ia":
        infinitivo = palabra[:-1] + "ti"
    elif palabra[-2:] == "ia":
        infinitivo = palabra[:-2] + "ti"
    elif palabra[-1] == "i":
        infinitivo = palabra[:-1] + "ti"
    elif palabra[-1] == "o":
        infinitivo = palabra[:-1] + "ti"
    else:
        infinitivo = None
    return infinitivo
    
# Creamos un conjunto para que los elementos añadidos no se repitan
infinitivos_artificiales = set()
for palabra in palabras_corpus:
    infinitivo = regla_infinitivo(palabra.lower())
    if infinitivo is not None:
        infinitivos_artificiales.add(infinitivo)
            

In [ ]:
#Vamos a instalar una librería para hacer peticiones web. 
!pip install requests

Vamos a utilizar esta librería para comprobar si la palabra construída según la regla de formación del infinitivo de verdad existe en lituano. El lituano es una lengua flexiva, por tanto varias categorías gramaticales pueden compartir las mismas terminaciones. Sin embargo, la única categoría gramatical que puede tener la terminación -ti es el infinitivo del verbo. Por tanto, si buscamos una palabra que acaba en -ti en un diccionario y nos devuelve un resultado, sabemos que la palabra existe (= es un verbo).

In [ ]:
# Vamos a usar expresiones regulares, código html, manejar caracteres de unicode 
# y hacer peticiones http (importamos librerías)
import re
import html
from unicodedata import normalize
import requests

# Urls del diccionario online del lituano
url = "http://www.lkzd.lki.lt/Zodynas/Zodziai.asp?txtZodis="
url2 = "http://www.lkzd.lki.lt/Zodynas/Straipsniui.asp?idr={}&id={}&idg=xxx&txz={}&txpz={}"


def elimina_sufijo_infinitivo(infinitivo):
    raiz = infinitivo[:-2] # Eliminamos la terminación de infinitivo "ti"

    # Si tras quitar la terminación la raíz termina en ciertos grupos de vocales, también hay que eliminarlas.
    # Ej., valgyti -> valg

    terminaciones = ('a', 'ą', 'e', 'ę', 'ė', 'i', 'į', 'y', 'uo', 'u', 'ų', 'ū')

    while True:
        if raiz.endswith(terminaciones):
            raiz = raiz[:-1]
        else:
            break

    return raiz


def construye_tiempo_verbal(infinitivo, sufijo):
    """Construye un tiempo verbal a partir del infinitivo y el sufijo del tiempo verbal deseado."""
    sufijo = sufijo[1:] # Eliminamos el guión
    raiz = elimina_sufijo_infinitivo(infinitivo)

    caracteres_para_quitar_de_raiz = sufijo
    while True:
        if len(caracteres_para_quitar_de_raiz) == 0:
            break

        # Quitamos las letras del final de raíz que coincidan con el principio del sufijo
        # Ej., dainuoti, -uoja --> dain + uoja (coinciden dos letras)
        if raiz.endswith(caracteres_para_quitar_de_raiz):
            return raiz.split(caracteres_para_quitar_de_raiz)[0] + sufijo
        else:
            caracteres_para_quitar_de_raiz = caracteres_para_quitar_de_raiz[:-1]

    return raiz + sufijo


def limpia_cadena(str):
    """Elimina todo lo que no sea la palabra o el sufijo."""
    if str.startswith("("):
        str = str.split(")")[1]

    str = str.strip(", ").split(",")[0].split(" ")[0]

    # Sustituimos letras que se interpretan mal (la letra 'ė' aparece como 'ë')
    str = str.replace('ë', 'ė')
    str = str.replace('þ', 'ž')
    str = str.replace('û́', 'ū')
    str = str.replace('è', 'č')
    return str


def quita_acentos(palabra):
    """Elimina los acentos (ej., dainúoti --> dainuoti)."""
    acentos = {
        "á": "a",
        "à": "a",
        "ã": "a",
        "é": "e",
        "è": "e",
        "ẽ": "e",
        "í": "i",
        "ì": "i",
        "ĩ": "i",
        "ó": "o",
        "ò": "o",
        "õ": "o",
        "ú": "u",
        "ù": "u",
        "ũ": "u",
        "ý": "y",
        "ỹ": "y",
        "ñ": "n"}

    # Unificamos los caracteres unicode para poderlos comparar
    palabra = normalize("NFC", palabra)
    for acento in acentos.keys():
        acento = normalize("NFC", acento)
        if acento in palabra:
            palabra = palabra.replace(acento, acentos[acento])
    return palabra

def es_verbo_lt(palabra):
    """
    Busca la palabra en el diccionario online y si es un verbo válido devuelve 
    una tupla con el infinitivo y la 3PSgPr
    """

    # Hacemos una petición para ver si la palabra existe como lema en el diccionario online
    response = requests.get(url + palabra)
    response_html = response.text

    # Si la respuesta contiene el lema (la palabra es válida), buscamos un patrón en el que figure el infinitivo
    # y recogemos unos parámetros que vamos a utilizar después
    regex_a_encontrar = "href=JavaScript:ActStr\('{0}','(\w+)','(\d+)','(\d+)'\)".format(palabra)
    patron = re.compile(regex_a_encontrar)
    resultados = patron.search(response_html)

    # Si hemos encontrado el patrón,'resultados' no es None, con el primer grupo regex (\d+) construimos la url2
    # en la que vamos a pedir la definición de la palabra
    if resultados:
        grupos = resultados.groups()
        base = grupos[0]
        palabra_id = grupos[1]
        idr = palabra_id[:2]

        if (base != palabra):
            return False

        response = requests.get(url2.format(idr, palabra_id, base, palabra))
        # Convertimos el texto en formato html en una cadena unicode válida (ej., &eacute; -> é)
        response_html = html.unescape(response.text)

        # Lo que nos interesa encontrar es el infinitivo y la 3PSgPr que son las formas básicas del verbo
        # que pueden aparecer de dos posibles maneras, por lo que tenemos que comprobar dos regex distintas
        regex_a_encontrar = "<div class=az><span class=g0>([^<]+)</span><span class=g6>([^<]+)</span>"
        patron = re.compile(regex_a_encontrar)
        resultados = patron.search(response_html)

        if resultados is None:
            regex_a_encontrar = "<div class=az><span class=g2>1 </span><span class=g11>([^<]+)</span><span class=g6>([^<]+)</span>"
            patron = re.compile(regex_a_encontrar)
            resultados = patron.search(response_html)

            if resultados is None:
                return False

        # En el diccionario la 3PSgPr puede venir indicada de dos maneras: como una palabra
        # (en el caso de que haya diferencias en la raíz) o como una terminación (en el caso de que
        # la raíz sea la misma que en el infinitivo
        grupos = resultados.groups()
        sufijos = grupos[1]

        # El acento en lituano es cambiante y variado, y viene indicado en los diccionarios. Sin embargo,
        # no se indica fuera del ámbito lexicográfico y debemos eliminarlo
        palabra = quita_acentos(palabra)
        sufijos = quita_acentos(sufijos)

        # Borramos todo lo que venga entre paréntesis
        sufijos = re.sub(r"\([^)]*\)", "", sufijos)

        # Si queda algún paréntesis abierto, es un caso en el que el html del diccionario viene
        # mal formado y no podemos interpretarlo
        if '(' in sufijos:
            return False

        try:
            presente, pasado = sufijos.strip(", ").split(",")[:2]
        except:
            return False

        infinitivo = limpia_cadena(palabra)
        presente = limpia_cadena(presente)
        pasado = limpia_cadena(pasado)

        # Si la 3PSgPr no viene indicada como una palabra, la tenemos que construir
        if presente.startswith("-"):
            presente = construye_tiempo_verbal(infinitivo, presente)

        # Si la 3PSgPr no viene indicada como una palabra, la tenemos que construir
        if pasado.startswith("-"):
            pasado = construye_tiempo_verbal(infinitivo, pasado)

        return (infinitivo, presente, pasado)
    else:
        return False
    
# Vamos a comprobar si los verbos construidos son verbos válidos en lituano y en el caso de que lo sean
# pedimos que nos devuelva el verbo en infinitivo y la tercera persona del singular del presente
# ya que estas dos formas pueden diferir en la raíz.
# Una vez que tenemos la lista de verbos, la guardamos en un fichero que luego leerá el bot.
fichero = open(corpusdir + "/coleccion_v.txt","w") 

for palabra in infinitivos_artificiales:
    verbo = es_verbo_lt(palabra)
    if verbo:
        fichero.write(verbo[0] + "," + verbo[1] + "," + verbo[2] + "\n") 

file.close() 
        